# Install Dependencies
---

In [57]:
# for installing packages using legacy code
!pip install setuptools

# The environment we will be working with
!pip install CityLearn

# For participant interactions (buttons)
!pip install ipywidgets

# To generate static figures
!pip install matplotlib
!pip install seaborn

# Provide standard RL algorithms
!pip install stable-StableBaselines3Wrapper
! pip install shimmy


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ERROR: Could not find a version that satisfies the requirement stable-StableBaselines3Wrapper (from versions: none)
ERROR: No matching distribution found for stable-StableBaselines3Wrapper


# Import Dependencies
---

In [61]:
import os
import numpy as np
import pandas as pd
import shutil
from stable_baselines3.sac import SAC
from citylearn.agents.rbc import BasicRBC
from citylearn.citylearn import CityLearnEnv
from citylearn.data import DataSet
from citylearn.reward_function import RewardFunction
from citylearn.utilities import read_json, write_json
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

In [59]:
SCHEMA_TEMPLATE = 'citylearn_challenge_2022_phase_1'
TARGET_DATASET_DIRECTORY = 'ev_test_data'
SCHEMA_FILEPATH = os.path.join(TARGET_DATASET_DIRECTORY, SCHEMA_TEMPLATE, 'schema.json')

# Create Test Dataset
---

In [71]:
# Copy an existing dataset
if os.path.isdir(TARGET_DATASET_DIRECTORY):
    shutil.rmtree(TARGET_DATASET_DIRECTORY)
else:
    pass

DataSet.copy(SCHEMA_TEMPLATE, TARGET_DATASET_DIRECTORY)
schema = read_json(SCHEMA_FILEPATH)

# add ev to buildings
CAPACITITES = [100.0, 60.0, 80.0, 40.0]
NOMINAL_POWERS = [9.0, 10.0, 8.0, 12.0]
EV_COUNTS = [1, 2, 3, 4, 5]
ARRIVAL_HOUR = (7, 10)
DEPARTURE_HOUR = (15, 18)
ARRIVAL_SOC = (0.2, 0.6)
DAYS = 365
HOURS = list(range(1, 25))*DAYS
HOURS = pd.DataFrame(HOURS, columns=['hour'])
HOURS['day'] = (HOURS.index/HOURS['hour'].max()).astype(int)

for b in schema['buildings']:
    ev_count = np.random.choice(EV_COUNTS)
    evs = []

    for i in range(ev_count):
        # define object
        schedule_filename = f"{b}_EV_{int(i+1)}.csv"
        ev = {
            "type": "citylearn.energy_model.ElectricVehicle",
            "schedule": schedule_filename,
            "autosize": False,
            "attributes": {
            "capacity": np.random.choice(CAPACITITES),
            "efficiency": 0.9,
            "capacity_loss_coefficient": 1e-05,
            "loss_coefficient": 0.0,
            "nominal_power": np.random.choice(NOMINAL_POWERS),
            "soc_minimum_limit": 0.1,
            "soc_maximum_limit": 0.8,
            "discharge_count_limit": 2
            }
        }
        evs.append(ev)

        # create schedule
        schedule = pd.DataFrame(columns=['arrival_hour', 'departure_hour', 'arrival_soc'])
        schedule['arrival_hour'] = np.random.choice(range(*ARRIVAL_HOUR), size=DAYS)
        schedule['departure_hour'] = np.random.choice(range(*DEPARTURE_HOUR), size=DAYS)
        schedule['arrival_soc'] = np.random.uniform(*ARRIVAL_SOC, size=DAYS)
        schedule['day'] = schedule.index
        schedule = schedule.merge(HOURS, on='day')
        schedule['availability'] = 0
        schedule.loc[(schedule['arrival_hour']<=schedule['hour']) & (schedule['departure_hour'] >= schedule['hour']), 'availability'] = 1
        schedule.loc[schedule['hour']!=schedule['arrival_hour'], 'arrival_soc'] = None
        schedule[['availability', 'arrival_soc']].to_csv(os.path.join(TARGET_DATASET_DIRECTORY, SCHEMA_TEMPLATE, schedule_filename), index=False)

    schema['buildings'][b]['electric_vehicles'] = evs

write_json(SCHEMA_FILEPATH, schema)

ImportError: ignored

In [70]:
!pip install pandas

# Test Simulation
---

## RBC

In [62]:
env = CityLearnEnv(SCHEMA_FILEPATH)
model = BasicRBC(env)
model.learn(episodes=10)
display(env.evaluate())

,cost_function,value,name,level
0,1 - load_factor,0.987430,District,district
1,average_daily_peak,1.149743,District,district
2,carbon_emissions,1.186277,District,district
3,cost,1.084738,District,district
4,electricity_consumption,1.248857,District,district
5,peak_demand,1.051783,District,district
6,ramping,1.162248,District,district
7,zero_net_energy,1.148490,District,district
8,electricity_consumption,1.184364,Building_1,building
9,zero_net_energy,1.117844,Building_1,building


## SB3 SAC

In [66]:
class CustomReward(RewardFunction):
    def __init__(self, env: CityLearnEnv):
        super().__init__(env)

    def calculate(self):
        reward_list = []

        for b in self.env.buildings:
            e = b.net_electricity_consumption[-1]
            ec = b.electrical_storage.capacity_history[0]
            es = b.electrical_storage.soc[-1]/ec
            reward = 0.0
            reward += -(1.0 + np.sign(e)*es)*abs(e) if ec > ZERO_DIVISION_CAPACITY else 0.0
            reward_list.append(reward)


        if self.env.central_agent:
            reward = [sum(reward_list)]
        else:
            reward = reward_list

        return reward

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
# train
episodes = 5
env = CityLearnEnv(SCHEMA_FILEPATH, central_agent=True)
env.reward_function = CustomReward(env) # set custom reward
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)
model = SAC('MlpPolicy', env, learning_starts=env.unwrapped.time_steps)
model.learn(total_timesteps=episodes*model.learning_starts)

# evaluate
observations = env.reset()
done = False

while not env.done:
    print(f'\rTime step: {env.time_step + 1}/{env.time_steps}', end=' '*50)
    actions, _ = model.predict(observations, deterministic=True)
    observations, reward, done, _ = env.step(actions)

display(env.evaluate())

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


KeyboardInterrupt: ignored